In [5]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [6]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixFactv1.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixFactv1.csv
day8_features_matrixFactv1.csv
day9_features_matrixFactv1.csv
test_features_matrixFactv1.csv


# Load Ip

In [7]:
# path = '/home/kai/data/kaggle/talkingdata/data/'
# train = pd.read_csv(path + 'train_cleaned_final.csv')
# print('loading training done')
# test = pd.read_csv(path + 'test_cleaned_final.csv')
# print('loading testing done')

# df_hour = pd.read_csv(path+'hourdistri.csv', index_col='Unnamed: 0')
# index = {}
# for day in ['day7', 'day8','day9']:
#     print(day)
#     index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
#     list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
#     list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 
#     df_dict[day]['ip'] = train.loc[index[day], 'ip'].values
# print('processing test!')
# df_dict['test']['ip'] = test['ip']


# just use categorical values that exist in both train and test
no improvement  
ip will not help

In [48]:
# def intersec_category(df_train_all, df_test_all, df, col_list):
#     for col in col_list:
#         print('processing: {}'.format(col))
#         train_index = set(df_train_all[col].value_counts().index)
#         test_index = set(df_test_all[col].value_counts().index)                 
#         inter_index = list(train_index.intersection(test_index))
#         print('train index length is: {}'.format(len(train_index)))
#         print('test index length is: {}'.format(len(test_index)))
#         print('intersection index length is: {}'.format(len(inter_index)))
#         index_map = pd.Series(inter_index, index=inter_index)
#         df.loc[:,col] = df[col].map(index_map).fillna(-1)
#     return df

# col_list = [ 'app', 'device', 'os', 'channel']
# df_train_all = pd.concat([df_dict[day_list[0]],df_dict[day_list[1]], df_dict[day_list[2]]])
# # for file in ['day7', 'day8', 'day9','test']: 
# for file in ['day7']:
#     temp = intersec_category(df_train_all, df_dict['test'], df_dict[file],col_list)

processing: app
train index length is: 556
test index length is: 417
intersection index length is: 372
processing: device
train index length is: 2438
test index length is: 1985
intersection index length is: 1437
processing: os
train index length is: 492
test index length is: 395
intersection index length is: 300
processing: channel
train index length is: 193
test index length is: 178
intersection index length is: 178


# define parameter

In [15]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 7,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 190,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.9, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 200,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(df_dict['test'].columns.values)

In [16]:
df_dict['day7'].columns.values

array(['ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_device_os_time2nextclick',
       'ip_app_device_os_time2nextclick',
       'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_device_os_countfromfuture', 'ip_app_device_os_countfromfuture',
       'ip_device_os_countfrompast', 'ip_app_device_os_countfrompast',
       'ip_device_os_lasttimediff', 'ip_app_device_os_lasttimediff',
       'matrixFact_user_iposdeviceapp_item_device',
       'matrixFact_user_iposdeviceapp_item_app',
       'matrixFact_user_iposdeviceapp_item_channel', 'app', 'channel',
       'device', 'os', 'hour', 'is_attributed'], dtype=object)

In [17]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['matrixFact_user_iposdeviceapp_item_device', 'matrixFact_user_iposdeviceapp_item_channel'])
feature_cols = list(set(feature_cols) - extra)

In [18]:
feature_cols

['ip_device_os_time2previousclick',
 'ip_day_hour_count',
 'ip_os_day_hour_count',
 'ip_app_day_hour_count',
 'ip_device_os_time2nextclick',
 'ip_app_device_os_countfromfuture',
 'ip_device_os_count',
 'device',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_os_lasttimediff',
 'os',
 'channel',
 'ip_device_os_countfrompast',
 'ip_device_os_countfromfuture',
 'ip_device_os_lasttimediff',
 'ip_app_device_os_time2previousclick',
 'app_day_hour_count',
 'app',
 'ip_device_os_mean',
 'ip_app_device_os_mean',
 'ip_app_device_os_countfrompast',
 'matrixFact_user_iposdeviceapp_item_app',
 'hour',
 'ip_app_os_day_hour_count',
 'ip_app_device_os_count']

In [19]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [20]:
for day in day_list:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_predtest_matrixFactv1_updatepara1.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    df_all = pd.concat([trainset, valset])
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_{}{}_val_{}_matrixFactv1_updatepara1.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)

    

concat_day8day9_val_day7_predtest_matrixFactv1_updatepara1.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.966965
[20]	valid_0's auc: 0.96945
[30]	valid_0's auc: 0.975163
[40]	valid_0's auc: 0.978476
[50]	valid_0's auc: 0.980015
[60]	valid_0's auc: 0.981104
[70]	valid_0's auc: 0.981947
[80]	valid_0's auc: 0.982346
[90]	valid_0's auc: 0.982706
[100]	valid_0's auc: 0.983024
[110]	valid_0's auc: 0.983212
[120]	valid_0's auc: 0.983362
[130]	valid_0's auc: 0.983459
[140]	valid_0's auc: 0.983599
[150]	valid_0's auc: 0.983721
[160]	valid_0's auc: 0.983813
[170]	valid_0's auc: 0.983837
[180]	valid_0's auc: 0.98388
[190]	valid_0's auc: 0.983906
[200]	valid_0's auc: 0.983949
[210]	valid_0's auc: 0.983978
[220]	valid_0's auc: 0.984008
[230]	valid_0's auc: 0.984074
[240]	valid_0's auc: 0.984089
[250]	valid_0's auc: 0.984129
[260]	valid_0's auc: 0.984152
[270]	valid_0's auc: 0.98418
[280]	valid_0's auc: 0.984202
[290]	valid_0's auc: 0.984232
[300]	valid_0's auc: 0.984253
[310]	valid_0's auc: 0.984264
[320]	valid_0's auc

KeyboardInterrupt: 

In [ ]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

# Training on All

In [29]:
###other
trainset = pd.concat([df_dict[day_list[0]],df_dict[day_list[1]], df_dict[day_list[2]]])
valset =df_dict[day_list[2]]

best_round = 1000

model = train_lightgbm(trainset, valset, feature_cols, categorical_feature, params, best_round)
gc.collect()
pred = model.predict(df_dict['test'][feature_cols])



start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[10]	valid_0's auc: 0.957823
[20]	valid_0's auc: 0.961944
[30]	valid_0's auc: 0.967703
[40]	valid_0's auc: 0.972667
[50]	valid_0's auc: 0.975523
[60]	valid_0's auc: 0.9773
[70]	valid_0's auc: 0.978444
[80]	valid_0's auc: 0.979037


KeyboardInterrupt: 

In [ ]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_ntree1000.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)


# Average

In [3]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixFactv1_appdevice.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixFactv1_appdevice.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixFactv1_appdevice.npy')

pred = (day7 + day8 + day9)/3

In [4]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixFactv1_appdevice.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
